# OCR recognition with Transformers (Training Phase)

#### important libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from math import ceil, floor

from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from datasets import load_metric

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
os.environ["WANDB_DISABLED"] = "True"

#### Import all the MNIST digits dataset

In [3]:


digits = [0,1,2,3,4,5,6,7,8,9]
path = 'C:\\Users\\A\\datasets\\MNIST - JPG - training\\'
train = []

for digit in digits:
    digit_path = path+str(digit)+'\*'
    for file in glob.glob(digit_path):
        # make label from filename
        filename = os.path.basename(file)
        #label = filename.split('_')[0]
        train.append([file, str(digit)])

df = pd.DataFrame(train, columns=['file_name', 'text'])

#### Create a sample of the dataset (not used here!)

In [1]:

#df['text'].value_counts()
#df2 = pd.DataFrame()
#for digit in digits:
#    df_sample = df[df['text'] == str(digit)].sample(n=300,random_state = 1)
#    df2 = df2.append(df_sample)

In [5]:
df2.text.value_counts()

0    300
1    300
2    300
3    300
4    300
5    300
6    300
7    300
8    300
9    300
Name: text, dtype: int64

#### Import all the Custom digits dataset

In [6]:
# custom label dataset
digits = [0,1,2,3,4,5,6,7,8,9]
digits_select = [0,2,3,5,7,8,9]
path = 'C:\\Users\\A\\datasets\\school_label_dataset\\extract_digits\\'

train = []

for digit in digits:
    digit_path = path+str(digit)+'\*'
    for file in glob.glob(digit_path):
    # make label from filename
        filename = os.path.basename(file)
        #label = filename.split('_')[0]
        train.append([file, str(digit)])

df3 = pd.DataFrame(train, columns=['file_name', 'text'])

In [15]:
df3['text'].value_counts()

1    319
3    300
4    268
2    265
6    256
8    254
7    237
0    230
5    190
9    156
Name: text, dtype: int64

#### split / train testset for training

In [11]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

train_df2, test_df2 = train_test_split(train_df, test_size=0.2)
# we reset the indices to start from zero
train_df2.reset_index(drop=True, inplace=True)
test_df2.reset_index(drop=True, inplace=True)

In [12]:
test_df

,file_name,text
0,C:\Users\A\datasets\school_label_dataset\extra...,7
1,C:\Users\A\datasets\school_label_dataset\extra...,0
2,C:\Users\A\datasets\school_label_dataset\extra...,5
3,C:\Users\A\datasets\school_label_dataset\extra...,3
4,C:\Users\A\datasets\school_label_dataset\extra...,7
...,...,...
490,C:\Users\A\datasets\school_label_dataset\extra...,1
491,C:\Users\A\datasets\school_label_dataset\extra...,7
492,C:\Users\A\datasets\school_label_dataset\extra...,3
493,C:\Users\A\datasets\school_label_dataset\extra...,7


#### Create encoding class, so that the encoder knows what to do

In [13]:


class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=1):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}

        return encoding

#### Load the pretrained processor and create two sets; train and valid

In [14]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='',
                           df=train_df2,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='',
                           df=test_df2,
                           processor=processor)

In [15]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 1782
Number of validation examples: 198


In [16]:
encoding = train_dataset[0]
for k,v in encoding.items():
    print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([3])


In [17]:
labels = encoding['labels']
print(labels)

labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print('Decoded Label:', label_str)

tensor([  0, 466,   2])
Decoded Label: 9


In [20]:
# print sample image
image = Image.open(train_dataset.root_dir + train_df['file_name'][0]).convert("RGB")
print('Label: '+train_df['text'][0])
image

Label: 4


#### Create metrics function, so this can be used to test the validation set

In [21]:

cer_metric = load_metric("cer")

def compute_metrics(pred):
    #print(pred)
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
    #print(list(pred_str))
    #print(type(label_str))
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    accuracy = accuracy_score(y_true=list(label_str[:]), y_pred=list(pred_str[:]))
    recall = recall_score(y_true=list(label_str[:]), y_pred=list(pred_str[:]), average = "weighted")
    precision = precision_score(y_true=list(label_str[:]), y_pred=list(pred_str[:]), average = "weighted")
    return {"cer": cer, "pred": pred_str,"labels": label_str,"accuracy": accuracy, "recall": recall, "precision":precision}

#### Load the pretrained model

In [22]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Create parameters (these have been changed a lot and this seems to give the best results)

In [23]:
# parameters

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = 10

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 10
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

#### Train the model with Seq2SeqTrainer

In [24]:
from transformers import default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=3,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=True, 
    output_dir=".",
    logging_steps=2,
    save_steps=50000,
    eval_steps=50,
    save_total_limit=1,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

trainer.train()

#Step	Training Loss	Validation Loss	Cer
#5000	0.449600	6.391628	2.151000
#10000	0.113700	9.725458	2.002444
#15000	0.291600	8.634416	1.493889
#20000	0.232700	9.673293	1.266778
#25000	0.333000	10.493820	1.084333
#30000	0.042500	10.242682	1.793444
#35000	0.004500	11.528788	2.001000

Using amp half precision backend
C:\Users\A\anaconda3\envs\tf-gpu\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1782
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1338
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: peterjans. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Cer,Pred,Labels,Accuracy,Recall,Precision
50,0.734300,0.790122,0.212121,"['4', '6', '6', '6', '4', '4', '5', '8', '1', '4', '3', '0', '6', '1', '8', '6', '4', '0', '2', '0', '4', '3', '8', '0', '2', '4', '1', '2', '4', '1', '4', '2', '4', '3', '6', '7', '1', '2', '2', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '2', '4', '3', '5', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '2', '6', '2', '6', '2', '0', '5', '4', '2', '1', '4', '8', '2', '4', '1', '3', '2', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '2', '5', '0', '4', '0', '4', '2', '4', '4', '6', '2', '5', '4', '2', '8', '2', '2', '2', '0', '8', '2', '6', '2', '2', '1', '1', '2', '1', '6', '4', '6', '7', '4', '2', '6', '5', '2', '4', '4', '8', '4', '8', '0', '5', '6', '2', '5', '5', '2', '2', '3', '3', '2', '4', '1', '2', '1', '5', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '2', '5', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '2', '2', '0', '0', '8', '5', '0', '4', '4', '3', '2', '4', '1', '8', '5', '8', '8', '5', '6', '5', '8']","['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '3', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '4', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '7', '2', '0', '0', '8', '5', '0', '4', '7', '3', '2', '4', '1', '8', '5', '8', '8', '9', '6', '3', '9']",0.787879,0.787879,0.823850
100,0.753000,0.751251,0.156566,"['7', '6', '6', '6', '6', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '6', '0', '7', '0', '1', '3', '8', '0', '8', '1', '1', '2', '1', '1', '7', '2', '6', '3', '6', '7', '1', '7', '7', '7', '1', '8', '6', '1', '2', '7', '6', '3', '3', '8', '7', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '6', '3', '2', '1', '8', '0', '8', '0', '1', '0', '1', '2', '1', '1', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '6', '6', '8', '7', '8', '0', '5', '6', '8', '5', '6', '2', '7', '3', '3', '2', '6', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '6', '2', '6', '2', '1', '6', '6', '5', '5', '0', '1', '5', '7', '2', '0', '0', '8', '5', '0', '4', '7', '3', '2', '1', '1', '8', '5', '8', '8', '9', '6', '3', '8']","['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '3', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '4', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4',

***** Running Evaluation *****
  Num examples = 198
  Batch size = 4
C:\Users\A\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "['4', '6', '6', '6', '4', '4', '5', '8', '1', '4', '3', '0', '6', '1', '8', '6', '4', '0', '2', '0', '4', '3', '8', '0', '2', '4', '1', '2', '4', '1', '4', '2', '4', '3', '6', '7', '1', '2', '2', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '2', '4', '3', '5', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '2', '6', '2', '6', '2', '0', '5', '4', '2', '1', '4', '8', '2', '4', '1', '3', '2', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '2', '5', '0', '4', '0', '4', '2', '4', '4', '6', '2', '5', '4', '2', '8', '2', '2', '2', '0', '8',

Trainer is attempting to log a value of "['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '3', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '4', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '7', '2', '0', '0', '8', '5', '0', '4', '7', '3', '2', '4', '1', '8', '5'

***** Running Evaluation *****
  Num examples = 198
  Batch size = 4
Trainer is attempting to log a value of "['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '8', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '7', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '9', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '0', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '2',

...

***** Running Evaluation *****
  Num examples = 198
  Batch size = 4
Trainer is attempting to log a value of "['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '1', '3', '4', '4', '2', '4', '6', '3', '3', '2', '7', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '0', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '7',

Trainer is attempting to log a value of "['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '3', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '4', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '7', '2', '0', '0', '8', '5', '0', '4', '7', '3', '2', '4', '1', '8', '5'

***** Running Evaluation *****
  Num examples = 198
  Batch size = 4
Trainer is attempting to log a value of "['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '3', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '0', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '7',

Trainer is attempting to log a value of "['1', '6', '6', '6', '4', '7', '5', '8', '1', '7', '3', '0', '6', '1', '8', '6', '4', '0', '9', '0', '4', '3', '8', '0', '3', '1', '1', '2', '4', '1', '7', '2', '4', '3', '6', '7', '1', '7', '7', '7', '4', '3', '4', '4', '2', '4', '6', '3', '3', '8', '3', '6', '3', '3', '5', '1', '3', '2', '0', '6', '2', '3', '0', '3', '6', '6', '7', '2', '7', '6', '2', '6', '2', '0', '9', '4', '2', '1', '1', '8', '2', '1', '1', '3', '7', '1', '3', '2', '8', '5', '1', '4', '3', '2', '1', '8', '0', '9', '0', '1', '0', '4', '2', '4', '4', '6', '7', '5', '4', '7', '8', '7', '7', '2', '0', '8', '2', '6', '2', '2', '1', '1', '7', '1', '6', '7', '6', '7', '1', '2', '6', '9', '7', '7', '4', '8', '7', '8', '4', '5', '6', '9', '5', '5', '2', '7', '3', '3', '2', '4', '1', '2', '1', '9', '8', '6', '3', '1', '3', '2', '1', '0', '2', '8', '7', '4', '2', '4', '2', '1', '4', '6', '5', '5', '0', '4', '5', '7', '2', '0', '0', '8', '5', '0', '4', '7', '3', '2', '4', '1', '8', '5'

TrainOutput(global_step=1338, training_loss=0.5739106673786281, metrics={'train_runtime': 1237.1423, 'train_samples_per_second': 4.321, 'train_steps_per_second': 1.082, 'total_flos': 4.730651079232979e+18, 'train_loss': 0.5739106673786281, 'epoch': 3.0})

#### specify the FineTune model path (Pytorch model from above) and load the processor and model of ViT

In [ ]:
model_path = r"C:\Users\A\Classification Aico\modelling\training\model\mnist_dataset_1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten") 
model = VisionEncoderDecoderModel.from_pretrained(model_path).to(device) 

#### Loop over all the testing set (created before) and generate the text; add these to a dictionary

In [ ]:
dict_ = dict()
for file in test_df.file_name:
    image = Image.open(file).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values.to(device))
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    dict_[file] = generated_text

# Create a pandas dataframe from a dictionary
df_model_test = pd.DataFrame.from_dict(dict_, orient='index').reset_index()

# Merge the outcome with the testing set for comparison
df_merge = pd.merge(df_model_test,test_df,left_on='index',right_on='file_name')

# Write to a csv file
df_merge.to_csv('test_acc_custom.csv',index=False)

In [ ]:
df_merge = pd.read_csv('test_acc_mnist.csv')

#### Create and print the classification report

In [ ]:

y_true = df_merge['text']
y_pred = df_merge['0']

In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred)
print(classification_report(y_pred,y_true))

#### Plot the confusion matrix

In [ ]:
ax = sns.heatmap(cf_matrix , annot=True, cmap='magma',fmt='g')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
report = classification_report(y_true, y_pred, output_dict=True)
report['accuracy']